In [27]:
import pandas as pd
df = pd.read_csv("../../data/battery_capacity_featured.csv")
df.head()

,cycle,capacity,temperature,capacity_fade_percent,capacity_diff,rolling_capacity_mean,degradation_rate
0,1,2.50,25.00,0.0,NaN,NaN,NaN
1,2,2.47,25.00,3.0,-0.03,2.485,3.0
2,4,2.40,26.00,10.0,-0.07,2.435,7.0
3,5,2.35,25.75,15.0,-0.05,2.375,5.0
4,6,2.30,27.00,20.0,-0.05,2.325,5.0


In [28]:
df = df.dropna()
x = df[["cycle", "temperature", "rolling_capacity_mean"]]
y = df["capacity_fade_percent"]
x.head(), y.head()
df

,cycle,capacity,temperature,capacity_fade_percent,capacity_diff,rolling_capacity_mean,degradation_rate
1,2,2.47,25.00,3.0,-0.03,2.485,3.0
2,4,2.40,26.00,10.0,-0.07,2.435,7.0
3,5,2.35,25.75,15.0,-0.05,2.375,5.0
4,6,2.30,27.00,20.0,-0.05,2.325,5.0


In [29]:
# adding train/test split(multi feature case earlier it was only one feature just cycle)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x,y, test_size=0.3, random_state=42
)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)

x_train shape: (2, 3)
y_train shape: (2,)


In [30]:
# yesterday it was y = a.slope + b today it will be fade = a1.cycle + a2.temperature + a3.rolling_mean + b ( 3 coeffiecient and 1 intercept)
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
model_lr.fit(x_train, y_train)
y_pred_lr = model_lr.predict(x_test)

In [31]:
#lets check what it learned also in rolling capacity mean there was one nan value so we used dropna in the starting
print("coefficients:", model_lr.coef_)
print("intercept:", model_lr.intercept_) 


coefficients: [ 3.7599482   0.93998705 -0.13786477]
intercept: -27.676978672738336


In [32]:
# creating evaluation function 
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
def evaluate_model(model_name, y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print("model:", model_name)
    print("rmse:", round(rmse,4))
    print("r2:", round(r2, 4))
    print("-"*30)

In [33]:
# now lets evaluate linear model 
evaluate_model("Linear Regression", y_test, y_pred_lr)

model: Linear Regression
rmse: 1.038
r2: 0.9569
------------------------------


In [34]:
#. Decision Tree
from sklearn.tree import DecisionTreeRegressor

model_dt = DecisionTreeRegressor(random_state=42)
model_dt.fit(x_train, y_train)

y_pred_dt = model_dt.predict(x_test)

In [35]:
evaluate_model("Decision tree", y_test, y_pred_dt)

model: Decision tree
rmse: 5.0
r2: 0.0
------------------------------


In [36]:
# overfitting detection 
print("Linear regression train r2:", model_lr.score(x_train, y_train))
print("Linear regression train r2:", model_lr.score(x_test, y_test))

print("Linear regression train r2:", model_dt.score(x_train, y_train))
print("Linear regression train r2:", model_dt.score(x_test, y_test))

Linear regression train r2: 1.0
Linear regression train r2: 0.9569036373820299
Linear regression train r2: 1.0
Linear regression train r2: 0.0
